# Context Managers

Context Managers are useful for managing shared resources and memory allocation.

An example of Context Manager can be seen with files:

In [2]:
# Create a file, do something with it and then close it
f = open("example.txt", 'w', encoding="utf-8")
f.write("Hello!")
f.close()

However, imagine that an exception happens when the file is opened and you're not able to reach the '.close()' line. In that file, the file object would never be closed. This is where Context Managers come in handy. To use a Context Manager with files, we use the 'with open(...)' statement:

In [3]:
with open("example.txt", 'w', encoding="utf-8") as f:
    f.write("Hello")
    # Here more operations can happen...
    
# Here, the file object 'f' is closed.

The last piece of code ensures that whatever happens, the file object is always closed. This is equivalent to say:

In [4]:
f = open("example.txt", 'w', encoding="utf-8")
try:
    f.write("Hello!")
    # ... more things can happen here
except Exception as e:
    raise e
finally:
    # The 'finally' block is always executed, no matter if an exception has happened
    f.close()

When opening and working with the file, these things are happening:
- First, the file is opened and the 'f' file object variable is created. Then, 'f' is returned.
- We do some operations on the file.
- Finally, the file is closed. 

The same behaviour can be implemented using a class with the '\_\_enter\_\_' and '\_\_exit\_\_' methods:

In [14]:
class File:
    
    def __init__(self, filename, method):
        # We define a constructor with the filename and the method ('w', 'r', 'a').
        
        # The file is opened:
        print("__init__ method")
        self._file = open(filename, method)
        
    def __enter__(self):
        # This is what happens when starting. 
        # In this case, it simply returns the file object
        
        print("__enter__ method")
        return self._file
    
    def __exit__(self, type, value, traceback): 
        # This is what happens when ending.
        # In this case, it simply closes the file.
        # We use 'type', 'value' and 'traceback' to handle exceptions
        
        print("__exit__ method")
        
        if type:  # print the exception info if captured
            print(f"Exception type captured by __exit__ is {type}.")
            print(f"Exception value captured by __exit__ is {value}.")
            print(f"Exception traceback captured by __exit__ is {traceback}.")
        
        self._file.close()
        

# We can use this class as always:
file_obj = File("example.txt", 'w')
print(file_obj)

# And we can use it like a Context Manager using 'with'
# it calls __init__ and after that __enter__.
# __enter__ returns the file object and it is stored at the variable 'f'
with File("example.txt", 'w') as f:  
    # Now, let's do something with the file. This happens between the 
    # __enter__ and the __exit__ methods.
    print("Working with the file...")
    f.write("Hello")
# When we reach this point, the __exit__ method is called. 

__init__ method
__init__ method
__enter__ method
Working with the file...
__exit__ method


The \_\_exit\_\_ method is executed even if an exception happens between it and the \_\_enter\_\_ method:

In [26]:
# Let's simulate a exception
with File("example.txt", 'w') as f:  # __init__ and __enter__
    raise FileNotFoundError("Simulated Exception")  # simulate a 'FileNotFoundError' exception
# At this point, __exit__ is still called


__init__ method
__enter__ method
__exit__ method
Exception type captured by __exit__ is <class 'FileNotFoundError'>.
Exception value captured by __exit__ is Simulated Exception.
Exception traceback captured by __exit__ is <traceback object at 0x00000191259EFF00>.


\_\_exit\_\_ can access all the information about the exception thanks to its parameters.

Sometimes, we want to continue the program despite of the exception. We can achieve this behaviour making \_\_exit\_\_ to always return a True boolean.

In [25]:
class File:
    
    def __init__(self, filename, method):
        # We define a constructor with the filename and the method ('w', 'r', 'a').
        
        # The file is opened:
        print("__init__ method")
        self._file = open(filename, method)
        
    def __enter__(self):
        # This is what happens when starting. 
        # In this case, it simply returns the file object
        
        print("__enter__ method")
        return self._file
    
    def __exit__(self, type, value, traceback): 
        # This is what happens when ending.
        # In this case, it simply closes the file.
        # We use 'type', 'value' and 'traceback' to handle exceptions
        
        print("__exit__ method")
        
        if type:  # print the exception info if captured
            print(f"Exception type captured by __exit__ is {type}.")
            print(f"Exception value captured by __exit__ is {value}.")
            print(f"Exception traceback captured by __exit__ is {traceback}.")
        
        self._file.close()
        
        # Let's make the program continue even if there is an Exception
        # (be careful using this, this is just an example)
        return True
        

# Let's simulate a exception
with File("example.txt", 'w') as f:  # __init__ and __enter__
    raise FileNotFoundError("Simulated Exception")  # simulate a 'FileNotFoundError' exception
# Here, __exit__ is still called


__init__ method
__enter__ method
__exit__ method
Exception type captured by __exit__ is <class 'FileNotFoundError'>.
Exception value captured by __exit__ is Simulated Exception.
Exception traceback captured by __exit__ is <traceback object at 0x00000191259EF4C0>.


# Creating Context Managers with Generators and Decorators

There's a way of creating Context Manager using Generators, Decorators and the *contextlib* library:

In [33]:
from contextlib import contextmanager

# Suppose we want to create a function that opens a file, returns it to the
# user code and whatever happens, it always closes the file. 
@contextmanager  # first, we decorate that function with '@contextmanager'
def open_a_file(filename, method):
    # The first thing that happens is that the function opens the file
    print(f"Opening the file: {filename}")
    file = open(filename, method)
    
    # Now we wanna return the file object to the user code. We use the 
    # 'yield' keywork because we want to mantain the internal state of
    # this function, that is, we're using a generator here
    print(f"Yielding the file object: {file}")
    yield file  # simply returns the file object
    
    # This line of code is executed when we finished the context, that is,
    # it behaves like the __exit__ method. No matter if an exception happens
    print(f"Closing the file object: {file}")
    file.close()
    print("File closed")
    
# Let's use this function. We use the 'with' statement again.
with open_a_file("example.txt", 'w') as f:  # opens the file and yields the file object
    print("Doing something with the file...")
    f.write("Hello")
    print("Finish doing something with the file")
# Here, the 'file.close()' line is executed

Opening the file: example.txt
Yielding the file object: <_io.TextIOWrapper name='example.txt' mode='w' encoding='cp1252'>
Doing something with the file...
Finish doing something with the file
Closing the file object: <_io.TextIOWrapper name='example.txt' mode='w' encoding='cp1252'>
File closed
